In [1]:
%pip install gspread oauth2client

Note: you may need to restart the kernel to use updated packages.


In [ ]:
import pandas as pd
from datetime import datetime
import io

In [1]:
import json
import pandas as pd
import numpy as np

# Path to the log file
file_path = "ajjas_logs.txt"  # Replace with your file path

# Read the log file
with open(file_path, "r") as file:
    ajjas_logs = file.readlines()

# Reverse the log entries to process from earliest to latest
log_entries = ajjas_logs[::-1]  # Reverse the order to process chronologically

# Initialize lists for columns
columns = ["timestamp", "a", "vid", "imei", "lat", "lng", "spd", "ts", "course", "ignition", "batVolt", "extVolt", "immobilizeStatus", "t"]
data = {col: [] for col in columns}

# Helper variables
current_timestamp = None

# Process each log entry
for entry in log_entries:
    try:
        # Check if the line is a timestamp
        if ":" in entry and len(entry.split(":")) == 3:
            # Extract the timestamp
            current_timestamp = entry.strip()
        else:
            # Parse the log line as JSON
            log_data = json.loads(entry.strip())
            
            # Extract fields
            a = log_data.get("a", np.nan)
            d = log_data.get("d", {})
            
            # Append data to the respective lists
            data["timestamp"].append(current_timestamp)
            data["a"].append(a)
            data["vid"].append(d.get("vid", np.nan))
            data["imei"].append(d.get("imei", np.nan))
            data["lat"].append(d.get("lat", np.nan))
            data["lng"].append(d.get("lng", np.nan))
            data["spd"].append(d.get("spd", np.nan))  # Speed value
            data["ts"].append(d.get("ts", np.nan))
            data["course"].append(d.get("course", np.nan))  # Course value
            data["ignition"].append(d.get("ignition", np.nan))  # Ignition status
            data["batVolt"].append(d.get("batVolt", np.nan))  # Battery Voltage
            data["extVolt"].append(d.get("extVolt", np.nan))  # External Voltage
            data["immobilizeStatus"].append(False)  # Immobilize status (defaulted to False)
            data["t"].append(d.get("t", np.nan))  # 't' value
    except (json.JSONDecodeError, KeyError) as e:
        # Add NaN for missing values if any error occurs
        for col in columns:
            if col not in ["timestamp", "immobilizeStatus"]:  # Skip timestamp and immobilizeStatus
                data[col].append(np.nan)
        data["timestamp"].append(current_timestamp)
        data["immobilizeStatus"].append(False)

# Create the DataFrame
df_ajjas_logs = pd.DataFrame(data)

# Save the DataFrame to a CSV file
output_file = "Ajjas_logs.csv"
df_ajjas_logs.to_csv(output_file, index=False)

# Notify the user where the file is saved
print(f"The processed data has been saved to {output_file}")


The processed data has been saved to Ajjas_logs.csv
